In [1]:
import netCDF4 as nc
from salishsea_tools import tidetools, nc_tools
import matplotlib.pyplot as plt
import numpy as np
import os
import re

%matplotlib inline

resultsDir='/data/eolson/MEOPAR/SS2DSOGruns/'
N2chl=1.600

TabError: inconsistent use of tabs and spaces in indentation (nc_tools.py, line 370)

In [ ]:
TS=nc.Dataset(os.path.join(resultsDir,'TS_uniform.nc'))

In [ ]:
# read z, T, S into CTD
CTD=np.loadtxt('/data/eolson/SOG/SOG-initial/ctd/SG-S3-2004-10-19.sog',skiprows=12,usecols=(1, 4))

In [2]:
nuts=np.loadtxt('/data/eolson/SOG/SOG-initial/stratogem_nuts/Nuts-S3-2004-10-19.sog', skiprows=12, usecols=(0,1,2))

NameError: name 'np' is not defined

In [3]:
# add z=0 at first row, repeating T,S values from next level
data=np.vstack((CTD[0,:],CTD))
data[0,0]=0.
# interpolate to T-points
data_f=(data[0:101,:]+data[1:102,:])/2
zs=data_f[:,0]
fluor=data_f[:,0]

NameError: name 'np' is not defined

In [ ]:
# Remove records with negative data values (typically -99.0 or
# -99999) because that indicates invalid data
data_qty=nuts[:,1]
mask = (data_qty >= 0.0)
data_records = len(data_qty[mask])
qty_clean = data_qty[mask]
depth_clean= nuts[:,0][mask]
# Calculate depth and quantity differences from field data for use
# in interpolation
del_depth = depth_clean[1:data_records] - depth_clean[0:data_records-1]
del_qty = qty_clean[1:data_records] - qty_clean[0:data_records-1]
# Interpolate quantity values at grid point depths
i_data = 1
NO=0.0*zs
for i_g in range(0, len(zs)):
    if (zs[i_g] > depth_clean[i_data]):
        while ((zs[i_g] > depth_clean[i_data]) and (i_data<data_records-1)):
            i_data = i_data + 1
    NO[i_g] = qty_clean[i_data-1] + del_qty[i_data-1] \
        * ((zs[i_g] - depth_clean[i_data-1]) \
        / del_depth[i_data-1])
print NO

In [ ]:
# Remove records with negative data values (typically -99.0 or
# -99999) because that indicates invalid data
data_qty=nuts[:,2]
mask = (data_qty >= 0.0)
data_records = len(data_qty[mask])
qty_clean = data_qty[mask]
depth_clean= nuts[:,0][mask]
# Calculate depth and quantity differences from field data for use
# in interpolation
del_depth = depth_clean[1:data_records] - depth_clean[0:data_records-1]
del_qty = qty_clean[1:data_records] - qty_clean[0:data_records-1]
# Interpolate quantity values at grid point depths
i_data = 1
Si=0.0*zs
for i_g in range(0, len(zs)):
    if (zs[i_g] > depth_clean[i_data]):
        while ((zs[i_g] > depth_clean[i_data]) and (i_data<data_records-1)):
            i_data = i_data + 1
    Si[i_g] = qty_clean[i_data-1] + del_qty[i_data-1] \
        * ((zs[i_g] - depth_clean[i_data-1]) \
        / del_depth[i_data-1])
print Si

In [ ]:
# create temp and sal arrays with correct dimensions
vecNO=np.reshape(NO,(101,1,1))
data_NO=np.tile(vecNO,(1,1,5,5))
vecSi=np.reshape(Si,(101,1,1))
data_Si=np.tile(vecSi,(1,1,5,5))
print data_Si.shape
vecP=0.33*np.reshape(data_f[:,1],(101,1,1))/N2chl
data_PHY=np.tile(vecP,(1,1,5,5))
data_PHY2=data_PHY
data_MYRI=data_PHY*1e-6
data_MICZ=data_MYRI
data_NH4=0.0*data_PHY+1.0
data_POC=data_PHY2/5.0*1e-6
data_DOC=data_POC/10.0
data_bSi=data_POC
data_POC=data_POC*7.6
data_DOC=data_DOC*7.6
print data_NH4[0,0,:,:]

In [ ]:
new_nuts=nc.Dataset('/data/eolson/MEOPAR/SS2DSOGruns/nuts_SOG5x5_S3-2014-10-19-WithMRubraMicroZooRemin.nc','w')
new_nuts.createDimension('y', 5)
new_nuts.createDimension('x', 5)
new_nuts.createDimension('deptht', 101)
new_nuts.createDimension('time_counter', None)

nc_tools.show_dimensions(TS)
nc_tools.show_dimensions(new_nuts)

In [ ]:
new_tc=new_nuts.createVariable('time_counter',float,('time_counter'),zlib=True)
new_tc.setncattr('units',TS.variables['time_counter'].units)
new_tc.setncattr('long_name',TS.variables['time_counter'].long_name)
new_tc[:]=TS.variables['time_counter']

print TS.variables['time_counter']
print new_nuts.variables['time_counter']

In [ ]:
new_z=new_nuts.createVariable('deptht',float,('deptht'),zlib=True)
new_z.setncattr('units',TS.variables['deptht'].units)
new_z.setncattr('long_name',TS.variables['deptht'].long_name)
new_z.setncattr('positive',TS.variables['deptht'].positive)
new_z[:]=zs

print TS.variables['deptht']
print new_nuts.variables['deptht']

In [ ]:
new_x=new_nuts.createVariable('x',float,('y','x'),zlib=True)
new_x.setncattr('units',TS.variables['x'].units)
new_x.setncattr('long_name',TS.variables['x'].long_name)
new_x[:]=TS.variables['x'][0:5,0:5]

print TS.variables['x']
print new_nuts.variables['x']

In [ ]:
new_y=new_nuts.createVariable('y',float,('y','x'),zlib=True)
new_y.setncattr('units',TS.variables['y'].units)
new_y.setncattr('long_name',TS.variables['y'].long_name)
new_y[:]=TS.variables['y'][0:5,0:5]

print TS.variables['y']
print new_nuts.variables['y']

In [ ]:
new_NO=new_nuts.createVariable('NO3',float,('time_counter','deptht','y','x'),zlib=True)
#new_Tem.setncattr('units',TS.variables['NO3'].units)
new_NO.setncattr('long_name','Nitrate')
new_NO.setncattr('coordinates',TS.variables['votemper'].coordinates)
new_NO[:,:,:,:]=data_NO
print TS.variables['votemper']
print new_nuts.variables['NO3']

In [ ]:
new_Si=new_nuts.createVariable('Si',float,('time_counter','deptht','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
new_Si.setncattr('long_name','Silicate')
new_Si.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_Si[:,:,:,:]=data_Si
print TS.variables['vosaline']
print new_nuts.variables['Si']

In [ ]:
new_NH4=new_nuts.createVariable('NH4',float,('time_counter','deptht','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
new_NH4.setncattr('long_name','Ammonium')
new_NH4.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_NH4[:,:,:,:]=data_NH4
print TS.variables['vosaline']
print new_nuts.variables['NH4']

In [ ]:
new_PHY=new_nuts.createVariable('PHY',float,('time_counter','deptht','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
new_PHY.setncattr('long_name','PHY')
new_PHY.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_PHY[:,:,:,:]=data_PHY
print TS.variables['vosaline']
print new_nuts.variables['PHY']

In [ ]:
new_PHY2=new_nuts.createVariable('PHY2',float,('time_counter','deptht','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
new_PHY2.setncattr('long_name','PHY2')
new_PHY2.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_PHY2[:,:,:,:]=data_PHY2
print TS.variables['vosaline']
print new_nuts.variables['PHY2']

In [ ]:
new_MYRI=new_nuts.createVariable('MYRI',float,('time_counter','deptht','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
new_MYRI.setncattr('long_name','MYRI')
new_MYRI.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_MYRI[:,:,:,:]=data_MYRI
print TS.variables['vosaline']
print new_nuts.variables['MYRI']

In [ ]:
new_MICZ=new_nuts.createVariable('MICZ',float,('time_counter','deptht','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
new_MICZ.setncattr('long_name','MICZ')
new_MICZ.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_MICZ[:,:,:,:]=data_MICZ
print TS.variables['vosaline']
print new_nuts.variables['MICZ']

In [ ]:
new_POC=new_nuts.createVariable('POC',float,('time_counter','deptht','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
new_POC.setncattr('long_name','POC')
new_POC.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_POC[:,:,:,:]=data_POC
print TS.variables['vosaline']
print new_nuts.variables['POC']

In [ ]:
new_DOC=new_nuts.createVariable('DOC',float,('time_counter','deptht','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
new_DOC.setncattr('long_name','DOC')
new_DOC.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_DOC[:,:,:,:]=data_DOC
print TS.variables['vosaline']
print new_nuts.variables['DOC']

In [ ]:
new_bSi=new_nuts.createVariable('bSi',float,('time_counter','deptht','y','x'),zlib=True)
#new_Sal.setncattr('units',TS.variables['vosaline'].units)
new_bSi.setncattr('long_name','bSi')
new_bSi.setncattr('coordinates',TS.variables['vosaline'].coordinates)
new_bSi[:,:,:,:]=data_bSi
print TS.variables['vosaline']
print new_nuts.variables['bSi']

In [ ]:
new_nuts.title="""SS2DSOG NO, Si, NH4, PHY, PHY2, MYRI, Z, DOC, POC, bSi initialization"""
new_nuts.institution="""
Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia"""
new_nuts.comment= """
Based on SG-S3-2004-10-19.sog and nuts-S3-2004-10-19.sog"""
new_nuts.reference= """
eolson: createIC_NutsPhy_SS2DSOG5x5_S3-2004-10-19-WithMRubraMicrozooRemin.ipynb"""

In [ ]:
nc_tools.show_dataset_attrs(new_nuts)

In [ ]:
new_nuts.close()

In [ ]:
TS.close()


In [ ]:
B=nc.Dataset('/data/eolson/MEOPAR/SS2DSOGruns/nuts_SOG5x5_S3-2014-10-19-WithMRubra.nc')
NO=B.variables['NO3']
Si=B.variables['Si']
PHY=B.variables['MYRI']
NH=B.variables['NH4']
plt.pcolormesh(NO[0,:,:,3]); plt.colorbar()

In [ ]:

plt.pcolormesh(Si[0,:,:,3]); plt.colorbar()

In [ ]:

plt.pcolormesh(NH[0,:,:,3]); plt.colorbar()

In [ ]:

plt.pcolormesh(PHY[0,:,:,3]); plt.colorbar()

In [ ]:
print B.variables

In [ ]:
B.close()